#### Thursday, February 29, 2024

[google/gemma-7b](https://huggingface.co/google/gemma-7b)

mamba activate kllmpr

This all runs. 

To download these models, we need to be authenticated on huggingface.

In [4]:
from huggingface_hub import login
login()


In [1]:
# only target the 4090 ...
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
# pip install accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM

In [3]:
model_ckpt = "google/gemma-7b"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

# 0.7s
# 20.2s

In [5]:
model = AutoModelForCausalLM.from_pretrained(model_ckpt, device_map="auto")

# 11.1s
# 20282 MiB VRAM

# 243m 34.6s

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

In [7]:
outputs = model.generate(**input_ids)

# 29.7s
# 20668 MiB VRAM

# 30.8s 

/home/rob/miniforge3/envs/kllmpr/lib/python3.11/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [8]:
print(tokenizer.decode(outputs[0]))

<bos>Write me a poem about Machine Learning.

I’m not a poet, but I’


Running the model on a GPU using different precisions.

Using torch.float16

In [1]:
# RESTART THE KERNEL BEFORE RUNNING THIS!
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b")
model = AutoModelForCausalLM.from_pretrained("google/gemma-7b", device_map="auto", torch_dtype=torch.float16)

input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids)
print(tokenizer.decode(outputs[0]))

# 28.1s
# 9520 MiB VRAM


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/home/rob/miniforge3/envs/kllmpr/lib/python3.11/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<bos>Write me a poem about Machine Learning.

I’m not a poet, but I’


Using torch.bfloat16

In [1]:
# RESTART THE KERNEL BEFORE RUNNING THIS!
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b")
model = AutoModelForCausalLM.from_pretrained("google/gemma-7b", device_map="auto", torch_dtype=torch.bfloat16)

# 5.6s
# 9278 MiB VRAM

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

In [ ]:
# Why does this fail??
outputs = model.generate(**input_ids)
print(tokenizer.decode(outputs[0]))

Quantized versions through bitsandbytes

Using 8-bit precision (int8)

In [1]:
# RESTART THE KERNEL BEFORE RUNNING THIS!
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_8bit=True)

In [2]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b")

In [3]:
model = AutoModelForCausalLM.from_pretrained("google/gemma-7b", quantization_config=quantization_config)

# 10.0s 
# 9858 Mib VRAM

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

In [5]:
outputs = model.generate(**input_ids)
print(tokenizer.decode(outputs[0]))

# 2.0s
# 12374 MiB VRAM


/home/rob/miniforge3/envs/kllmpr/lib/python3.11/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<bos>Write me a poem about Machine Learning.

I’m not a poet, but I’


Using 4-bit precision

In [1]:
# RESTART THE KERNEL BEFORE RUNNING THIS!
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_4bit=True)

In [2]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b")

In [3]:
model = AutoModelForCausalLM.from_pretrained("google/gemma-7b", quantization_config=quantization_config)

# 10.0s
# 6734 MiB VRAM

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

In [5]:
outputs = model.generate(**input_ids)
print(tokenizer.decode(outputs[0]))

# 0.6s
# 7680 MiB VRAM


/home/rob/miniforge3/envs/kllmpr/lib/python3.11/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/rob/miniforge3/envs/kllmpr/lib/python3.11/site-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


<bos>Write me a poem about Machine Learning.

<bos><bos><bos><bos><bos><bos><bos><bos><bos><bos>
